# Chapter 7. 함수 데코레이터와 클로저

## 7.1 데커레이터 기본 지식
데커레이터는 다른 함수를 인수로 받는 콜러블(데커레이트된 함수)이다. 데커레이터는 데커레이트된 함수에 어떤 처리를 수행하고, 함수를 반환하거나 함수를 다른 함수나 콜러블 객체로 대체한다.

예시처럼 decoreate라는 이름의 데커레이터가 있다면
    
```python
@decorate
def target():
    print('running target()')
```
위 코드는 다음과 같이 작동한다
```python
def target():
    print('running target()')

target = decorate(target)
```
두 가지 코드는 동일하다. 두 코드를 실행하면 target은 원래의 target() 함수를 가리키는 것이 아니고 decorate(target)이 반환한 함수를 가리키게 된다. 

In [4]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')

In [8]:
target()    # deco함수의 inner()가 실행된다.

running inner()


In [7]:
target      # deco함수의 inner()를 가리키고 있음

<function __main__.deco.<locals>.inner()>

## 7.2 파이썬이 데커레이터를 실행하는 시점


In [14]:
# example 7-2. register.py
registry = []

def register(func):
    print(f'running register({func}!!)')      # 데코레이터가 실행될 때마다 실행된다.
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')

def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

running register(<function f1 at 0x105f00ca0>!!)
running register(<function f2 at 0x105de08b0>!!)


In [4]:
main()

running main()
registry -> [<function f1 at 0x106751dc0>, <function f2 at 0x106751e50>]
running f1()
running f2()
running f3()


In [12]:
!python Ch7/register.py

running register(<function f1 at 0x1053e8160>)
running register(<function f2 at 0x1053e81f0>)
running main()
registry -> [<function f1 at 0x1053e8160>, <function f2 at 0x1053e81f0>]
running f1()
running f2()
running f3()


In [16]:
from Ch7 import register

register.registry

[<function Ch7.register.f1()>, <function Ch7.register.f2()>]

## 7.3 데커레이터로 개선한 전략 패턴

In [22]:
# example 7-3. promotion 데커레이터로 채운 promos 리스트

promos = []

def promotion(anything):
    promos.append(anything)
    return anything

@promotion
def fidelity(order):
    """충성도 포인트가 1000점 이상인 고객에게 5% 할인"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    """20개 이상의 동일 상품을 구입하면 10% 할인"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order(order):
    """10종류 이상의 상품을 구입하면 전체 구매 상품의 7% 할인"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

def best_promo(order):
    """최대로 할인해 줄 금액을 반환한다."""
    return max(promo(order) for promo in promos)


In [23]:
promos

[<function __main__.fidelity(order)>,
 <function __main__.bulk_item(order)>,
 <function __main__.large_order(order)>]

## 7.4 변수 범위 규칙

In [24]:
# 지역 및 전역 변수를 읽는 함수
def f1(a):
    print(a)
    print(b)

f1(3)

3


NameError: name 'b' is not defined

In [26]:
# 지역변수가 없으면 전역변수를 참조한다.
b = 6
f1(3)

3
6


In [32]:
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9

f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [35]:
# 지역변수가 있지만 전역변수를 가져오고 싶다면 global 키워드를 사용한다.
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9

f3(3)

3
6


In [39]:
# BYTE CODE
from dis import dis

print(dis(f1),end='========================================\n')
print(dis(f2),end='========================================\n')
print(dis(f3),end='========================================\n')

  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  4           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE
None========================================
  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  4           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  5          16 LOAD_CONST               1 (9)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE
None====================================

## 7.5 클로저(Closure)
클로져 != 익명함수
클로저는 함수 본체에서 정의하지 않고 참조하는 비전역(nonlocal)변수를 포함한 확장 범위를 가진 함수다. 익명 함수여부는 중요하지 않고, 함수 본체 외부에 정의된 비전역 변수에 접근할 수 있다는 것이 중요하다.  
아래의 예제에서 `avg()`함수가 점차 증가하는 일련의 값의 평균을 계산한다고 가정했을 때, `avg()`함수는 `series`변수를 참조한다. `series`변수는 `avg()`함수의 본체에서 정의되지 않았다. 따라서 `avg()`함수는 `series`변수를 포함한 확장 범위를 가진다. 이런 함수를 클로저라고 한다.

```python
>>> avg(10)
10.0
>>> avg(11)
10.5
>>> avg(12)
11.0
```


In [40]:
class Average:
    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)

In [44]:
avg = Average()

print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0
